# Local Search

<ul id="top">
<li><a href="#Imports"> 
             Imports</a></li>
<li><a href="#Description-of-the-approach"> 
             Description of the approach</a></li>
</ul>

## Imports

In [1]:
using LinearAlgebra
using Plots 
using OrdinaryDiffEq
using Optim

include("src/IRK8.jl")
include("src/3body.jl");

## Description of the approach

Our task is to find different three body choreographies. We will pose it as an optimization problem. The search space is the space of all $u(\tau)$ curves that satisfy the equations of motion of Newton's universal law of gravitation. The objective function will determine the "choreographyness" of a curve, and a local search will be performed using said function.

## Encoding the candidate solutions

A solution to the ODEs $u$ can be represented by its initial state $u_0$. The dimensionality of the search space can be reduced in many ways, such as taking translational symmetry into account. An in-depth explanation can be found in the thesis. The initial state is encoded in two different ways, by $\psi_3$ and by $\psi_5$:
\begin{align*} 
\psi_3(a, b, \gamma) = (-2&a, 0, a, b, a, -b, \\
    &0, \frac{\rho}{-2\sqrt3} \sin(\gamma), \rho \cos(\gamma), \frac{\rho}{\sqrt3 \sin(\gamma)}, -\rho \cos(\gamma), \frac{\rho}{\sqrt3 \sin(\gamma)})
\end{align*}
, where $\rho = \sqrt{3b^2 + a^2}$
\begin{align*}
\psi_5(a, b, \alpha, \beta, \gamma_2, \gamma_3) &= (3a, b, 0, -b, -3a, b, \\
    -&\lambda \cos(\alpha)\cos(\gamma_2) - \lambda \sin(\alpha)\cos(\gamma_3), - \lambda \cos(\alpha)\sin(\gamma_2) - \lambda \sin(\alpha)\sin(\gamma_2), \\
    &\lambda \cos(\alpha)\cos(\gamma_2), \lambda \cos(\alpha)\sin(\gamma_2), \lambda \sin(\alpha)\cos(\gamma_3), \lambda \sin(\alpha)\sin(\gamma_3))
\end{align*}

We will implement $\psi_3$ as `get_u3`:

In [2]:
getRho2(x2, y2) = 1/(2*abs(y2)) + 2/sqrt(9*x2*x2 + y2*y2) - 0.5

function get_u3(x2, y2, gamma, rho2 = Inf)
    if rho2 == Inf
        rho2 = getRho2(x2, y2)
    end
    rho = sqrt(rho2)
    vx2 = rho * cos(gamma)
    vy2 = rho * sin(gamma)/sqrt(3)
    x1 = -2*x2
    y1 = 0
    vx1 = 0
    vy1 = -2*vy2
    
    x3 = x2
    y3 = -y2
    vx3 = -vx2
    vy3 = vy2
    
    return [3x2, y2, 0., -2y2, -3x2, y2, vx1, vy1, vx2, vy2, vx3, vy3, 0.0]
end

get_u3 (generic function with 2 methods)

We will implement $\psi_5$ as `get_u5`:

In [3]:
function get_u5(x2, y2, alpha, gamma2, gamma3, rho2 = Inf)
    if rho2 == Inf
        rho2 = getRho2(x2, y2)
    end
    (sinalpha,cosalpha) = sincos(alpha)
    (singamma2,cosgamma2) = sincos(gamma2)
    (singamma3,cosgamma3) = sincos(gamma3)
    
    vx2aux = cosalpha*cosgamma2
    vy2aux = cosalpha*singamma2
    
    vx3aux = sinalpha*cosgamma3
    vy3aux = sinalpha*singamma3
    
    vx1aux = -vx2aux - vx3aux 
    vy1aux = -vy2aux - vy3aux
    
    lambda = sqrt(2 * rho2 / (1 +  vx1aux^2 + vy1aux^2))
    vx1 = lambda*vx1aux
    vy1 = lambda*vy1aux  
    vx2 = lambda*vx2aux
    vy2 = lambda*vy2aux  
    vx3 = lambda*vx3aux 
    vy3 = lambda*vy3aux
    return [3x2, y2, 0., -2y2, -3x2, y2, 
            vx1, vy1, vx2, vy2, vx3, vy3, 0.] 
end

get_u5 (generic function with 2 methods)

## Objective functions

We will define multiple objective functions, each of which designed to find a specific kind of choreography. These objective functions will be composed of a "trigger" function $g$ and a "callback" function $h$.

The objective functions are all of the following form:
$$
\mathcal{F}(q) = \min_{\tau \in G_q} h(q(\tau))
$$
where
$$
G_q = \{\tau : g(q(\tau)) = 0\}
$$

The idea here is that our algorithm will evaluate $g$ throughout the curve $q$, and if at some point $q(\tau)$ we have that the trigger function $g$ returns 0 it will mean that the bodies are in some sort of special configuration. Then, we evaluate $h$ on the points where $g$ returned 0 to determine its coreographyness. 

### Isosceles function

The first trigger function that we will look at detects that the bodies form an isosceles configuration with $g$:

$$
g_{\text{iso}}(u(\tau)) = (q_{23} \cdot q_{31} - q_{31} \cdot q_{12})(q_{32} \cdot q_{12} - q_{12} \cdot q_{23})
$$


In [4]:
function g_iso(u, t, integrator)
    dot12 = u[1]*u[3]+u[2]*u[4]
    dot23 = u[3]*u[5]+u[4]*u[6]
    dot31 = u[5]*u[1]+u[6]*u[2]
    return (dot23-dot31)*(dot31-dot12)
end

g_iso (generic function with 1 method)

Callback function $h_{\text{iso}}$:

\begin{equation*}
h_{\text{iso}}(u(\tau)) = \left\{
        \begin{array}{ll}
            h_{\text{iso2}}(u(\tau)) & \quad \text{if} \quad \nu(u(\tau)) \geq 0 \\
            h_{\text{iso3}}(u(\tau)) & \quad \text{else}
        \end{array}
    \right.
\end{equation*}

where 

$$
h_{\text{iso2}}(u(\tau)) = ((x_{23} - x_{12})\dot x_{23} +(y_{23} - y_{12})\dot y_{23})^2 + (\left\lVert\dot q_{12}\right\rVert^2 - \left\lVert\dot q_{31}\right\rVert^2)^2
$$

$$
h_{\text{iso3}}(u(\tau)) = ((x_{31} - x_{23})\dot x_{31} +(y_{31} - y_{23})\dot y_{31})^2 + (\left\lVert\dot q_{12}\right\rVert^2 - \left\lVert\dot q_{23}\right\rVert^2)^2
$$

$$
\nu(u(\tau)) = 2 (q_{31} \cdot q_{12}) - q_{12} \cdot q_{23} - q_{23} \cdot q_{31}
$$

In [8]:
h_iso2(u) = ((u[3]-u[1])*u[9]+(u[4]-u[2])*u[10])^2 + (u[7]^2+u[8]^2-u[11]^2-u[12]^2)^2
h_iso3(u) = ((u[5]-u[3])*u[11]+(u[6]-u[4])*u[12])^2 + (u[7]^2+u[8]^2-u[9]^2-u[10]^2)^2

function h_iso(u, u0)
    dot12 = u[1]*u[3]+u[2]*u[4]
    dot23 = u[3]*u[5]+u[4]*u[6]
    dot31 = u[5]*u[1]+u[6]*u[2]
    return (dot23-dot31) <= abs(dot31-dot12) ? h_iso2(u) : h_iso3(u)
end

h_iso (generic function with 1 method)

### Supersymmetry function
$g_{\text{col}}$ will detect collinear configurations, i. e. when all three bodies align.

In [9]:
g_col(u, t, integrator) = u[1]*u[4]-u[2]*u[3]

g_col (generic function with 1 method)

$h_{\text{col}}$

In [10]:
h_col2(u) = (u[3]-u[1])^2 + (u[4]-u[2])^2 + (u[11]-u[7])^2 + (u[12]-u[8])^2
h_col3(u) = (u[5]-u[3])^2 + (u[6]-u[4])^2 + (u[7]-u[9])^2 + (u[8]-u[10])^2

h_col(u, u0) = abs(u[5]) >= abs(u[1]) ? h_col2(u) : h_col3(u)

h_col (generic function with 1 method)

### Asymmetry function

g

In [11]:
g_asy = g_iso

g_iso (generic function with 1 method)

h

In [12]:
c(ax, ay, bx, by) = (ax*bx+ay*by)^2/((ax^2+ay^2)*(bx^2+by^2))

h_asy(u, u0) =  ((c(u[5], u[6], -u[3], -u[4]) - 
                c(u0[1], u0[2], -u0[5], -u0[6])))^2 + 
                ((c(u[1], u[2], -u[5], -u[6]) - 
                c(u0[3], u0[4], -u0[1], -u0[2])))^2 +  
                ((c(u[3], u[4], -u[1], -u[2]) - 
                c(u0[5], u0[6], -u0[3], -u0[4])))^2 +  
                (u[11]^2 + u[12]^2 - u0[7]^2 - u0[8]^2)^2 +
                (u[7]^2 + u[8]^2 - u0[9]^2 - u0[10]^2)^2^2 +
                (u[9]^2 + u[10]^2 - u0[11]^2 - u0[12]^2)^2

h_asy (generic function with 1 method)

## Indexing
Brief explanation

## Local search

In [86]:
function evaluate(u0, p, f, Tmin, Tmax, g, h, index=nothing)
    # Get all t for which g(q(t)) = 0
    tspan = (0,Tmax)
    alg = Vern9()
    prob = ODEProblem(f,u0,tspan,p)
    cb = ContinuousCallback(
        g,
        integrator->nothing,
        integrator->nothing,
        save_positions = (true, false),
        rootfind = true
        )
    
    sol=solve(prob, alg, saveat=Tmax,
              adaptive=false,dt=0.01,callback=cb)
    
    # If it is unindexed then return the argmin h(q(t))
    if index == nothing
        gpoints = [[t, u, h(u, u0)] for (t, u) in zip(sol.t, sol.u) if t > Tmin]
        if length(gpoints) == 0
            T_res, uT_res, gT_res = 0., u0, Inf
        else
            _, minind = findmin(map(first, gpoints))
            T_res, uT_res, gT_res = gpoints[minind]
        end
    # If it is indexed then return corresponding h(q(t))
    else
        if length(sol.t) < index+1
            T_res, uT_res, gT_res = 0., u0, Inf
        elseif sol.t[index+1] < Tmin 
            T_res, uT_res, gT_res = 0., u0, Inf
        else
            T_res, uT_res, gT_res = sol.t[index+1], sol.u[index+1], h(sol.u[index+1], u0)
        end
    end
    
    return T_res, uT_res, gT_res
end

evaluate (generic function with 2 methods)

In [97]:
g = g_iso
h = h_iso
Tmin = 1.
Tmax = 20.
p = nothing
odef = ThreeBodyODEGlobalTR!
u0 = get_u3(1, 1, 1);

In [99]:
evaluate(u0, p, odef, Tmin, Tmax, g, h)

(20.0, [-0.1266967411614648, -3.705715308104575, -0.780216510565657, -1.664560727400334, 0.906913251727123, 5.370276035504908, -0.5440262244154597, -0.13936157145425335, 0.7425313618810393, -0.12868166740861917, -0.19850513746557766, 0.26804323886287396, 31.37982121054939], 0.6010524642551067)

In [ ]:
function rand_u3(xa, xz, ya, yz, ca, cz)
    x2 = rand()*(xa-xz)+xa
    y2 = rand()*(ya-yz)+ya
    c = rand()*(ca-cz)+ca
    return [x2, y2, c]
end

function rand_u5(xa, xz, ya, yz, ca, cz)
    #TODO
end

In [108]:
function objective_iso(val)
    rho2 = getRho2(val[1], val[2])
    if rho2 < 0
       return Inf
    end
    u0 = get_u(val[1], val[2], val[3], rho2)
    T, uT, gT = evaluate(u0, p, odef, Tmin, Tmax, g_iso, h_iso) 
    return gT
end

function objective_col(val)
    rho2 = getRho2(val[1], val[2])
    if rho2 < 0
       return Inf
    end
    u0 = get_u(val[1], val[2], val[3], rho2)
    T, uT, gT = evaluate(u0, p, odef, Tmin, Tmax, g_col, h_col) 
    return gT
end

function objective_asy(val)
    # TODO
end

objective_asy (generic function with 1 method)

In [109]:
objective = objective_iso
get_u = get_u3
rand_u = rand_u3

LoadError: invalid redefinition of constant objective

Aldatu hau

In [10]:
ez exekutatu

[][1]

try
    cond = true
    i = 1
    while cond
        x2 = rand()*(xmax-xmin)+xmin
        y2 = rand()*(ymax-ymin)+ymin
        gamma = rand()*(cmax-cmin)+cmin

        balioak = [x2,y2,gamma]
        #print(i, ": ", balioak, "\n")

        if objective(balioak) < Inf
            result = nothing
            try 
                result = optimize(helburu, balioak, NelderMead(), 
                              Optim.Options(g_tol = 1e-8, iterations = 256))
            catch e
                if isa(e, InterruptException)
                    break
                end
                #print("Errorea\n")
                continue
            end
            if result.minimum < min_g
                minimizer = result.minimizer
                berria_da = instert_undiscovered(koreografiak, minimizer, dist2)
                if berria_da
                    #print("Aurkituta: ", minimizer, "\n")
                    io = open(path, "a")
                    for elem in minimizer
                        write(io, elem)
                    end
                    close(io)
                else
                    #println("Errepikatuta.")
                end
            end
        end

        i = i+1
        cond = i <= maxiters && length(koreografiak) < maxlistsize
    end
    
catch err
    println(err)
finally
    println("Bilaketa amaituta.")
end

LoadError: syntax: extra token "exekutatu" after end of expression

In [11]:
u0 = initialize(koreografiak[1]...)

T, uT, gT = evaluate(u0, p, odef, Tmin, Tmax, g, h) 
visualize(u0, odef, p, 0., T, 256, 16)

LoadError: UndefVarError: koreografiak not defined